# Import required libraries and define the required functions

In [11]:
import cv2 as cv
import numpy as np
import urllib.request
from matplotlib import pyplot as plt
# from numba import jit
from ultralytics import YOLO

# import Models
model = YOLO('./models/yolov8n-seg.pt')
# model = YOLO('./models/yolov8x-seg.pt')
# model = YOLO('./models/yolov8s-seg.pt')
# model = YOLO('./models/yolov8m-seg.pt')
# model = YOLO('./models/yolov9c-seg.pt')
# model = YOLO('./models/yolov9e-seg.pt')
# model = YOLO('./models/yolo11n-seg.pt')
# model = YOLO('./models/yolo11x-seg.pt')
# model = YOLO('./models/yolo11s-seg.pt')
# model = YOLO('./models/yolo11m-seg.pt')


# Setting up focal length, Image number and baseline

In [12]:
img = 15
focal_length = 694
baseline = 15 // 100

# Required Functions

In [13]:
# @jit(nopython=True)
def downScale(image, n):
    modifiedImage = np.floor_divide(image, 2**n)
    return modifiedImage

# @jit(nopython=True)
def upScale(image, n):
    modifiedImage = np.multiply(image, 2**n) 
    return modifiedImage.astype(np.uint8)

def disparity_to_depth(disparity_map):
    disparity_map[disparity_map <= 0] = 0.1
    depth_map = (focal_length * baseline) / disparity_map

    return depth_map

# @jit(nopython=True)
def brightnessUP(image, b_level):
    return image + b_level

def contrastup(img, gamma):
    modified_img = np.array(np.clip(pow(img / 255.0, gamma) * 255.0, 0, 255), dtype=np.uint8)
    modified_img = np.clip(modified_img + 20, 0, 255)
    return modified_img

def scalelinear(img):
    max_intensity = np.max(img)
    min_intensity = np.min(img)
    modified = np.array(np.clip((img -min_intensity) * (255 / (max_intensity-min_intensity)), 0,  255), dtype=np.uint8)
    return modified

def segment_color(img, probability):
    temp = np.copy(img)
    results = model.predict(img, conf=probability)
    if len(results) > 0:
        for result in results:
            for mask, box in zip(result.masks.xy, result.boxes):
                points = np.int32([mask])
                cv.fillPoly(temp, points, 1)
    return temp - img

def viewImages(original, modified, text): 
    plt.figure(figsize=(9,9)) 

    plt.subplot(1,2,1) 
    plt.imshow(original) 
    plt.title('Orignal') 
    plt.axis('off') 

    plt.subplot(1,2,2) 
    plt.imshow(modified) 
    plt.title(text) 
    plt.axis('off') 
    plt.savefig('./maps/'+text+'.png', dpi=300)
    plt.tight_layout()


# For testing purposes, import stored images.

In [ ]:
l_img_name = './updated/L_'+str(img)+'0.png'
r_img_name = './updated/R_'+str(img)+'0.png'

left_img = cv.imread(l_img_name, cv.IMREAD_GRAYSCALE)
right_img = cv.imread(r_img_name, cv.IMREAD_GRAYSCALE)
viewImages(left_img, right_img, "temp")

left_color_img = cv.imread(l_img_name, cv.IMREAD_COLOR)
right_color_img = cv.imread(r_img_name, cv.IMREAD_COLOR)
viewImages(left_color_img, right_color_img, "temp")

# Getting images using ESP Cameras

In [15]:
left_url = 'http://192.168.205.216/cam-hi.jpg'
right_url = 'http://192.168.205.172/cam-hi.jpg'

left_img_resp=urllib.request.urlopen(left_url)
right_img_resp=urllib.request.urlopen(right_url)

left_imgnp=np.array(bytearray(left_img_resp.read()),dtype=np.uint8)
right_imgnp=np.array(bytearray(right_img_resp.read()),dtype=np.uint8)

left_color_img = cv.imdecode(left_imgnp,-1)
right_color_img = cv.imdecode(right_imgnp,-1)

left_img = cv.cvtColor(left_color_img, cv.COLOR_BGR2GRAY)
right_img = cv.cvtColor(right_color_img, cv.COLOR_BGR2GRAY)

# Getting images using USB cameras

In [ ]:
cap_left = cv.VideoCapture(0)
cap_right=cv.VideoCapture(1)

rect_left, left_img = cap_left.read()
rect_right, right_img = cap_right.read()

# Stereo Processing Algorithms

In [17]:
# Use SAD Algorithm for calculate disparities ---------------------------------------------------------------
# @jit(nopython=True)
def compute_disparity_SAD(left_img, right_img, block_size, disparities):
    h, w = left_img.shape
    disparity = np.zeros((h, w), np.uint8)
    half_block = block_size // 2

    for y in range(half_block, h - half_block):
        for x in range(half_block, w - half_block):
            left_block = left_img[y - half_block:y + half_block + 1,
                                       x - half_block:x + half_block + 1]

            min_sad = block_size ** 2 * 255
            best_disparity = 0

            for d in range(disparities):
                x_right = x - d

                if x_right - half_block >= 0:
                    right_block = right_img[y - half_block:y + half_block + 1,
                                                 x_right - half_block:x_right + half_block + 1]

                    sad = np.sum(np.abs(left_block.astype(np.int32) - right_block.astype(np.int32)))

                    if sad < min_sad:
                        min_sad = sad
                        best_disparity = d

            disparity[y, x] = best_disparity

    return disparity

# Use SSD Algorithm for calculate disparities ---------------------------------------------------------------
# @jit(nopython=True)
def compute_disparity_SSD(left_img, right_img, block_size, max_disparity):
    h, w = left_img.shape

    disparity = np.zeros((h, w), np.uint8)

    half_block = block_size // 2

    for y in range(half_block, h - half_block):
        for x in range(half_block, w - half_block):
            left_block = left_img[y - half_block:y + half_block + 1,
                                       x - half_block:x + half_block + 1]

            min_ssd = block_size ** 2 * 255
            best_disparity = 0

            for d in range(max_disparity):
                x_right = x - d

                if x_right - half_block >= 0:
                    right_block = right_img[y - half_block:y + half_block + 1,
                                                 x_right - half_block:x_right + half_block + 1]

                    ssd = np.sum((left_block.astype(np.int32) - right_block.astype(np.int32)) ** 2)

                    if ssd < min_ssd:
                        min_ssd = ssd
                        best_disparity = d
            disparity[y, x] = best_disparity

    return disparity

# Use NCC Algorithm for calculate disparities ---------------------------------------------------------------
# @jit(nopython=True)
def compute_disparity_NCC(left_img, right_img, block_size, max_disparity):

    h, w = left_img.shape

    disparity_map = np.zeros((h, w), np.float32)

    half_block = block_size // 2

    for y in range(half_block, h - half_block):
        for x in range(half_block, w - half_block):
            left_block = left_img[y - half_block:y + half_block + 1, x - half_block:x + half_block + 1]

            left_mean = np.mean(left_block)

            best_ncc = -1
            best_disparity = 0

            for d in range(max_disparity):
                x_right = x - d

                if x_right - half_block >= 0:
                    right_block = right_img[y - half_block:y + half_block + 1, x_right - half_block:x_right + half_block + 1]

                    right_mean = np.mean(right_block)

                    numerator = np.sum((left_block - left_mean) * (right_block - right_mean))
                    left_norm = np.sum((left_block - left_mean) ** 2)
                    right_norm = np.sum((right_block - right_mean) ** 2)
                    denominator = np.sqrt(left_norm * right_norm)

                    if denominator > 0:
                        ncc_score = numerator / denominator

                        if ncc_score > best_ncc:
                            best_ncc = ncc_score
                            best_disparity = d

            disparity_map[y, x] = best_disparity

    return disparity_map

# YOLO object detection

In [16]:
def segmentation_disparity(img, disparity_map):
    results = model(img)    
    masks = results[0].masks.data    
    img_with_squares = img.copy()
    disparity_h, disparity_w = disparity_map.shape
    count = 0
    for i, mask in enumerate(masks):
        count += 1
        mask_np = mask.cpu().numpy()        
        mask_binary = (mask_np > 0.5).astype(np.uint8)        
        mask_binary_resized = cv.resize(mask_binary, (disparity_w, disparity_h), interpolation=cv.INTER_NEAREST)        
        masked_disparity = disparity_map * mask_binary_resized        
        non_zero_disparity_values = masked_disparity[mask_binary_resized == 1]
        
        if len(non_zero_disparity_values) > 0:
            plt.imshow(masked_disparity, cmap='jet')
            plt.savefig(f'./maps/disparity{count}.png', dpi=300)
            avg_disparity = np.mean(non_zero_disparity_values)
        else:
            avg_disparity = 0
        x, y, w, h = cv.boundingRect(mask_binary_resized)
        
        color = (0, 255, 0)
        cv.rectangle(img_with_squares, (x, y), (x+w, y+h), color, 2)
        cv.putText(img_with_squares, f"Avg Disparity: {avg_disparity:.2f}", 
                   (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img_with_squares

def disparity_to_depth(avg_disparity, focal_length, baseline):
    if avg_disparity == 0:
        return 0
    depth = (focal_length * baseline) / avg_disparity
    return depth

def segmentation_depth(img, disparity_map):    
    results = model(img)
    masks = results[0].masks.data
    img_with_squares = img.copy()
    disparity_h, disparity_w = disparity_map.shape
    
    count = 0
    for i, mask in enumerate(masks):
        count += 1
        mask_np = mask.cpu().numpy()
        
        mask_binary = (mask_np > 0.5).astype(np.uint8)
        
        mask_binary_resized = cv.resize(mask_binary, (disparity_w, disparity_h), interpolation=cv.INTER_NEAREST)
        
        masked_disparity = disparity_map * mask_binary_resized
        
        non_zero_disparity_values = masked_disparity[mask_binary_resized == 1]
        
        if len(non_zero_disparity_values) > 0:
            avg_disparity = np.mean(non_zero_disparity_values)
        else:
            avg_disparity = 0

        avg_depth = disparity_to_depth(avg_disparity, focal_length, baseline)

        x, y, w, h = cv.boundingRect(mask_binary_resized)
        
        color = (0, 255, 0)
        cv.rectangle(img_with_squares, (x, y), (x+w, y+h), color, 2)
        cv.putText(img_with_squares, f"Avg Depth: {avg_depth:.2f}m", 
                   (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return img_with_squares

# Depth Estimation

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(150,150))

# ---------------------SAD Algorithm------------------------------------
disparity = compute_disparity_SAD(left_img, right_img, 5, 32)
img_disparity = segmentation_disparity(left_color_img, disparity)
img_depth = segmentation_depth(left_color_img, disparity)

axs[0, 0].imshow(img_disparity)
axs[0, 0].axis('on')
axs[0, 0].set_title("SAD-disparity")

axs[0, 1].imshow(img_depth)
axs[0, 1].axis('on')
axs[0, 1].set_title("SAD-depth")

# ---------------------SSD Algorithm------------------------------------
# disparity = compute_disparity_SSD(left_img, right_img, 5, 32)
# img_disparity = segmentation_disparity(left_color_img, disparity)
# img_depth = segmentation_depth(left_color_img, disparity)

# axs[0, 1].imshow(img_disparity)
# axs[0, 1].axis('on')
# axs[0, 1].set_title("SSD-disparity")

# axs[1, 1].imshow(img_depth)
# axs[1, 1].axis('on')
# axs[1, 1].set_title("SSD-depth")

# ---------------------NCC Algorithm------------------------------------
# disparity = compute_disparity_NCC(left_img, right_img, 5, 32)
# img_disparity = segmentation_disparity(left_color_img, disparity)
# img_depth = segmentation_depth(left_color_img, disparity)

# axs[0, 2].imshow(img_disparity)
# axs[0, 2].axis('on')
# axs[0, 2].set_title("NCC-disparity")

# axs[1, 2].imshow(img_depth)
# axs[1, 2].axis('on')
# axs[1, 2].set_title("NCC-depth")

# ---------------------CV StereoBM Algorithm------------------------------------
# stereo = cv.StereoBM_create(numDisparities=32, blockSize=5)
# disparity_sb = stereo.compute(left_img, right_img)

# img_disparity = segmentation_disparity(left_color_img, disparity)
# img_depth = segmentation_depth(left_color_img, disparity)

# axs[0, 3].imshow(img_disparity)
# axs[0, 3].axis('on')
# axs[0, 3].set_title("CV_BM-disparity")

# axs[1, 3].imshow(img_depth)
# axs[1, 3].axis('on')
# axs[1, 3].set_title("CV_BM-depth")

plt.tight_layout()
plt.savefig('./updated_maps/RealTimeDepthSAD.png', dpi=300)
plt.show()
 
